In [1]:
import seaborn as sns 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fnmatch import fnmatch

%load_ext autoreload
%autoreload 2

from src.modules.probabilities import Probabilities
import modules.generator as g
from modules.transition import Transition
from src.modules.solutionAlgorithms import SolutionAlgorithms
from src.modules.app import App
from src.modules import constants as con




In [2]:
df = pd.read_pickle("/usr/app/data/tmp/viInputDf.pkl")
df_states = pd.read_pickle("/usr/app/data/tmp/viDFStates.pkl")
sol = SolutionAlgorithms()
cont = sol.performStandardVI(df, df_states["s_obj"].to_dict())

FileNotFoundError: [Errno 2] No such file or directory: '/usr/app/data/tmp/viInputDf.pkl'

In [10]:
from modules.state import State
g.constructDecisions(State(0,4.0,0,15.5,0.0,0.0))

(0.0,0.0,0)[]
(0.0,0.0,1)[7]
(0.2,0.0,0)[]
(0.2,0.0,1)[4, 7]
(0.4,0.0,0)[]
(0.4,0.0,1)[4, 7]
(0.6,0.0,0)[]
(0.6,0.0,1)[4, 7]
(0.8,0.0,0)[]
(0.8,0.0,1)[4, 7]
(1.0,0.0,0)[]
(1.0,0.0,1)[4, 7]
(1.2,0.0,0)[1]
(1.2,0.0,1)[1, 4, 7]
(1.4,0.0,0)[1]
(1.4,0.0,1)[1, 4, 7]
(1.6,0.0,0)[1]
(1.6,0.0,1)[1, 4, 7]
(1.8,0.0,0)[1]
(1.8,0.0,1)[1, 4, 7]
(2.0,0.0,0)[1]
(2.0,0.0,1)[1, 4, 7]
(2.2,0.0,0)[1]
(2.2,0.0,1)[1, 4, 7]
(0.0,0.2,0)[]
(0.0,0.2,1)[4, 7]
(0.0,0.4,0)[]
(0.0,0.4,1)[4, 7]
(0.0,0.6,0)[]
(0.0,0.6,1)[4, 7]
(0.0,0.8,0)[]
(0.0,0.8,1)[4, 7]
(0.0,1.0,0)[]
(0.0,1.0,1)[4, 7]
(0.0,1.2,0)[]
(0.0,1.2,1)[4, 7]
(0.0,1.4,0)[]
(0.0,1.4,1)[4, 7]
(0.0,1.6,0)[]
(0.0,1.6,1)[4, 7]
(0.0,1.8,0)[]
(0.0,1.8,1)[4, 7]
(0.0,2.0,0)[]
(0.0,2.0,1)[4, 7]
(0.0,2.2,0)[]
(0.0,2.2,1)[4, 7]


,d_key,s_key,d_obj
0,"(0.0,0.0,0)","(0,4.0,0,15.5,0.0,0.0)","Decision [(0.0,0.0,0)]"
2,"(0.2,0.0,0)","(0,4.0,0,15.5,0.0,0.0)","Decision [(0.2,0.0,0)]"
4,"(0.4,0.0,0)","(0,4.0,0,15.5,0.0,0.0)","Decision [(0.4,0.0,0)]"
6,"(0.6,0.0,0)","(0,4.0,0,15.5,0.0,0.0)","Decision [(0.6,0.0,0)]"
8,"(0.8,0.0,0)","(0,4.0,0,15.5,0.0,0.0)","Decision [(0.8,0.0,0)]"
10,"(1.0,0.0,0)","(0,4.0,0,15.5,0.0,0.0)","Decision [(1.0,0.0,0)]"
24,"(0.0,0.2,0)","(0,4.0,0,15.5,0.0,0.0)","Decision [(0.0,0.2,0)]"
26,"(0.0,0.4,0)","(0,4.0,0,15.5,0.0,0.0)","Decision [(0.0,0.4,0)]"
28,"(0.0,0.6,0)","(0,4.0,0,15.5,0.0,0.0)","Decision [(0.0,0.6,0)]"
30,"(0.0,0.8,0)","(0,4.0,0,15.5,0.0,0.0)","Decision [(0.0,0.8,0)]"


In [ ]:
print(df_states[(~df_states.index.to_series().isin(df["s_key"])) & (df_states["s_obj"].apply(lambda s: (not s.get_isTerminal()) & (s.get_V_TA() == 1) & (s.get_B_L() > 3.9)))])
print(len(df_states[(~df_states.index.to_series().isin(df["s_key"]))]))
print(df.loc[(df["s_key"].apply(lambda k: fnmatch(k, "(1,5.0,1,20.5,0.1,0.1)")))])
df_states.loc['(2,3.0,0,0.0,0.0,0.0)']

In [ ]:
#df_states.head()
df_states.loc['(1,1.0,0,0.0,0.0,0.0)']
df.loc[(df["s_key"].apply(lambda k: fnmatch(k, "(?,1.?,0,0.0,0.0,0.0)")))] # TODO check states should not be filtered

In [ ]:
print(cont["cont"].describe())
cont.loc[(cont.cont.astype(float)<= -2150.220000),["s_key","d_key","cont"]]

cont.loc[(cont["s_key"].apply(lambda k: fnmatch(k, "(0,?.?,0,0.0,0.?,0.1)"))) & \
         (cont["d_key"].apply(lambda k: fnmatch(k, "(0.?,0.?,0)"))) & \
         (cont["cont"] == 0.02),["s_key","d_key"]]

In [ ]:
# (1) Derive Probabilties for transition


np.random.seed(14)
prob = Probabilities()

p = prob.getProbabilities(1*con.tau*60)

#sum(p["p"]) # == 1 !


In [ ]:
from src.modules import constants as con
# Construct ex_info data frame for all ts
df_p = pd.DataFrame()

for t in np.arange(0, 1440, con.tau*60):
    df_p = df_p.append(prob.getProbabilities(t))
    
df_p.tail()

In [ ]:
# (2) Derive state space


import time
start = time.process_time()
# your code here    
sts = g.constructStates()
print(time.process_time() - start)
sts.describe()

In [ ]:
sts.head()
len(sts[(sts["V_TA"] == 0) & (sts["t"] == 2) & (sts["B_L"] >= 40) & (sts["D"] == 0)]) ##120
len(sts[(sts["V_TA"] == 0) & (sts["t"] == 10) & (sts["B_L"] >= 40)]) ##120

In [ ]:
# (3) Derive decision space
from tqdm import tqdm
dec = {}

for s in tqdm(sts["obj"].head(1)):
    dec[s.getKey()] = g.constructDecisions(s)

#print(sts.iloc[0])
#print(dec)
print(pd.DataFrame([x.__str__() for x in dec[sts.loc[0, "obj"].getKey()]]))

In [ ]:
# Perform transitions
np.random.seed(14)

prob = Probabilities()
ex_inf = prob.getProbabilities(300).loc[0,:]
t = Transition(None, sts.loc[0, "obj"], dec[sts.loc[0,"obj"].getKey()][-1], ex_inf["p"], ex_inf[0:3])


print(sts.loc[0, "obj"])
print(dec[sts.loc[0,"obj"].getKey()][-1])
print(ex_inf)
print()
print(t.get_s_d().__str__())

In [3]:
a = App()
a.valueIteration()

FileNotFoundError: [Errno 2] No such file or directory: '/usr/app/src/logs/20210607_1109_app.log'

In [ ]:
#df.loc[(df["t"] == 0) & (df["s_obj"].apply(lambda x: x.get_B_L() == 4.2)) & (df["d_obj"].apply(lambda d: d.get_x_trip() == 1.0)),~df.columns.isin(['s_obj', 'd_obj', "tr_obj"])]
df.loc[(df["t"] == 0) & (df["d_obj"].apply(lambda d: d.get_x_t() == 1.0)),~df.columns.isin(['s_obj', 'd_obj', "tr_obj"])]['trpln'].astype(int).describe()
df.loc[ (df["s_obj"].apply(lambda x: (x.get_D() == 0.0) )) ,~df.columns.isin(['s_obj', 'd_obj', "tr_obj"])]

In [19]:
x = pd.DataFrame({"a":[1,2], "b": [2,3], "c": [2,3]})
x.append(pd.DataFrame({"a":["a","b"], "c": ["c","d"]}))

,a,b,c
0,1,2.0,2
1,2,3.0,3
0,a,NaN,c
1,b,NaN,d
